# Mad-Hatter Bot iterator script combined with semi-manual finetuning


In [ ]:
#Make sure to run this cell as its vital for the rest of the script to run.
from ratelimit import limits, sleep_and_retry
import itables.interactive
from itables import show
import datetime
import os
from functools import lru_cache
from BaseHaas import Bot
import dateutil.relativedelta
import ipywidgets as widgets
import pandas as pd
from haasomeapi.enums.EnumMadHatterIndicators import EnumMadHatterIndicators
from haasomeapi.enums.EnumMadHatterSafeties import EnumMadHatterSafeties
from haasomeapi.HaasomeClient import HaasomeClient
from ipywidgets import interact
from haasomeapi.enums.EnumErrorCode import EnumErrorCode

import configserver
import csv
from  botsellector import BotSellector
from botdb import BotDB, InteractiveBT
def calculate_ticks(start_date,end_date):
    diff = end_date-start_date
    secs = diff.total_seconds()/60
    return int(secs)
opts= BotDB().get_mh_bots()
botsellector = widgets.Dropdown(options=opts,value=opts[0][1],description='Select Bot',disabled=False,)
files = BotDB().get_csv_files()
filessellector = widgets.Dropdown(options=[x for x in files],value=files[2],description='Config file',disabled=False,)       
start_date=widgets.DatePicker(value=(datetime.date.today() + dateutil.relativedelta.relativedelta(days=-9)))
end_date=widgets.DatePicker(value=datetime.date.today())
bd = BotDB()


In [ ]:
# After running this block you will see Bot Selector, config file sellector, start_date and end_date sllector.
# Select a bot from list to backtest,
# Select config file from list to use. bots.csv should generally be uses as it contains over 700 configs.

# The latter you only need to change start_date to set backtesting begin date, no need to touch end_date.
# when selecting backtesting starting date, the best practice is to set it to the most recent trend
# change, i.e. the moment when pricemovement changed direction from down to uptrend.
# You only need to select a bot, a file and date from drop down menus where and move on to the next cell
display(botsellector), display(filessellector)
ticks = interact(calculate_ticks, start_date=start_date, end_date=end_date)

In [ ]:
# This cell gets down to business - begins backtesting stage.
# You will only see a list of updating backtesting results for now for reference.
# Backtesting stage execution time heavily relies on how long the backtest you have selected.
# If chosen bots.csv file,expect it to take around 30-40 minutes on 2-5 day backtesting interval.
# And up to two hours or longer if backtesting interval is set to weeks or month.

# Make sure to also run this cell after the previous one.
# It will throw an error once run on a rose color background.
# Just ignore that for now and enjoy backtestign results coming in.
# Once completed,  a star sign on the left hand side of this cell located in "In [*]:" changed to a number.
#  also below this cell, after backtestign results it will say "Backtesting Stage Complete"
bd = BotDB()
configs = bd.read_csv(filessellector.value)
results = bd.iterate_csv(configs, botsellector.value, ticks.widget.result)
print('Backtesting Stage Complete')

In [ ]:
# Run this cell as well. It performs sorting and cleaning operations on the results from above.
# Here will also be an error printed. Ignore that too.

results.sort_values(by='roi', ascending=False, inplace=True)
cols = results.columns.tolist()
# print(cols) 
cols = ['roi','interval', 'signalconsensus', 
        'fcc', 'resetmiddle', 'allowmidsells', 
        'matype', 'rsil', 'rsib', 'rsis', 'bbl',
        'devup', 'devdn', 'macdfast', 'macdslow', 
        'macdsign', 'trades']
results.reset_index()
results = results[cols]


In [ ]:
# This cell displays results in a more meaningful table-fashion manner for you to analyze.
# You have an ability to select how many configurations are displayed in dropdown above the table.
# In general you only need to care for the first row and Roi row.
# In the first row is bot configuration number. Roi is backtesting results
# To apply a bot configuration to the selected bot, type its number Below the table and click
# "Apply config" button for the bot to be configured according and backtested for you to see.
# Once applied, go to Haas interface, select that bot one more time by mouseclicking it
# if not in full screen, to see changes to its configuration.
# You should see the configuraion backtesting results, trades and so on once backtest is complete.
# There is no limit to he number of configurations you can apply to your bot, so take your time
# To analyzer the best ones in Bot Analyzer in Haas interface to make sure that it's trades align 
# with your expectations before going to the next stage.
# Once this cell run, you may have to scroll up a bit to see the table, so make sure to do that 
# its not at sight.
# Backtesting results may slightly vary from what is in the table, usually by not much.

import itables.interactive
from itables import show
import itables.options as opt
results.reset_index()
try:
    results.drop(['Unnamed: 0', ],axis=1,inplace=True)
except:
    pass
opt.lengthMenu = [10, 20, 50, 100]
show(results)
results_sellector = widgets.Dropdown(
    options= [x for x in results.index],
    value=results.index[0],
    description='Botlist',
    disabled=False,
)
def configure_bot_from_csv_by_id(index):
        bot = botsellector.value
        
        config = configs.loc[index]
        bd.setup_bot_from_csv(bot, config)
        bt = bd.bt_bot(bot,int(ticks.widget.result))


from IPython.display import display
button = widgets.Button(description="Apply config")
output = widgets.Output()
r_s = widgets.IntText(
    value=0,
    min=0,
    step=1,
    description='Bot Num:',
    disabled=False
)
display(r_s,button,output)

def on_button_clicked(b):
    configure_bot_from_csv_by_id(r_s.value)
    with output:
        print("Bot Config has been applied. Backtesting results available in Haas")
   

button.on_click(on_button_clicked)


In [ ]:

class InteractiveBT(Bot):
    def __init__(self):
        Bot.__init__(self)
        
    @sleep_and_retry
    @limits(calls=3, period=2)
    def return_botlist(self):
        bl = self.c().customBotApi.get_all_custom_bots().result
        botlist = [x for x in bl if x.botType == 15]
        # print(botlist)
        return botlist
    

    def identify_bot(self):
        botlist = self.return_botlist()
        while True:
            botlist2 = self.return_botlist()
            lists = zip(botlist,botlist2)
            for x in lists:
                c= self.compare_indicators(x[0],x[1])
                if c == False:
                    return x[1]
    @sleep_and_retry
    @limits(calls=3, period=2)
    def monitor_bot(self,bot, ticks):
        botlist = self.return_botlist()
        for b in botlist:
            if b.guid == bot.guid:
                c = self.compare_indicators(bot,b)
                if c == True:
                    pass
                elif c == False:
                    bot = self.bt_mh_on_update(b, ticks)
                return bot
           
    def compare_indicators(self, bot, bot1):
            # print(bot.rsi, '\n',bot1.rsi)
        rsi = bot.rsi.items() == bot1.rsi.items()
        bbands = bot.bBands.items() == bot1.bBands.items()
        macd = bot.macd.items() == bot1.macd.items()
        interval = bot.interval == bot1.interval
        if rsi == True and bbands == True and macd == True and interval == True:
            return True
        else:
            # print('bot not alike')
            return False
    
    def identify_changed_param(self,bot,bot1):
        diff = {}
        param_list = ['rsi','bBands','macd']
        for param in param_list:
            diff[param] = { k : bot.param[k] for k in set(bot1.param) - set(bot.param) }
            if len(diff)>0:
                return diff
            else:
                return None
        


    @sleep_and_retry
    @limits(calls=3, period=2)
    def bt_mh_on_update(self, bot, ticks):

        bt = self.c().customBotApi.backtest_custom_bot(
            bot.guid,
            int(ticks)
        )
        if bt.errorCode != EnumErrorCode.SUCCESS:
            print("bt", bt.errorCode, bt.errorMessage)
        else:
            # print(bt.result.roi)
            # print(bt.errorCode, bt.errorMessage)
            return bt.result
        




In [ ]:
ibt = InteractiveBT()
bot = ibt.identify_bot()
semi_auto_tuning = [bot]

def on_stop_button(b):
    return False
    
stopbutton = widgets.Button(description="STOP AutoBT")
while True:
    bot = ibt.monitor_bot(bot, ticks.widget.result)
    if semi_auto_tuning[-1].roi != bot.roi:
        semi_auto_tuning.append(bot)
#     print(bot.rsi)
#     print(bot.macd)
#     print(bot.bBands)
    if stopbutton.on_click(on_stop_button) == False:
        break

In [ ]:
# sortedresults = sorted(semi_auto_tuning,key=lambda x: x.roi, reverse=True)
df = pd.DataFrame([{'ROI':b.roi,'Bot':b} for b in semi_auto_tuning]) 
df.sort_values(by='ROI', ascending=False, inplace=True)
df.reset_index()
show(df)

def configure_bot_from_botobject(index):      
        config = df.Bot.iloc[index]
        print(config)
        bd.setup_bot(bot, config)
        bot = bd.bt_bot(bot,int(ticks.widget.result))
        
button = widgets.Button(description="Apply config")
output = widgets.Output()
r_s = widgets.IntText(
    value=0,
    min=0,
    step=1,
    description='Bot Num:',
    disabled=False
)
display(r_s,button,output)

def on_button_clicked(b):
    configure_bot_from_botobject(r_s.value)
    with output:
        print("Bot Config has been applied. Backtesting results available in Haas")
   

button.on_click(on_button_clicked)


In [ ]:
bot = bd.bt_bot(botsellector.value,1)
setup = bd.setup_bot(bot,botsellector.value)
= bd.bt_bot(bot,int(ticks.widget.result))